In [1]:
import random

import numpy as np
import pandas as pd

data1 = pd.read_csv('/opt/ml/submission/6065.csv')
data2 = pd.read_csv('/opt/ml/submission/6030.csv')
data3 = pd.read_csv('/opt/ml/submission/5997.csv')
data4 = pd.read_csv('/opt/ml/submission/5996.csv')
data5 = pd.read_csv('/opt/ml/submission/5975.csv')
data6 = pd.read_csv('/opt/ml/submission/sy.csv')

In [115]:
data1_str = data1.iloc[0]['PredictionString']

In [116]:
data1_str = data1_str.split(' ')

In [117]:
data1_str = np.array(data1_str, int)

In [2]:
len(data1_str)

In [119]:
submission = pd.read_csv('/opt/ml/submission/sample_submission.csv', index_col=None)

In [3]:
from tqdm.notebook import tqdm

submission = pd.read_csv('/opt/ml/submission/sample_submission.csv', index_col=None)
submission

for i in tqdm(range(len(data1))):
    file_name = data1.iloc[i]['image_id']
    
    data1_str = data1.iloc[i]['PredictionString']
    data2_str = data2.iloc[i]['PredictionString']
    data3_str = data3.iloc[i]['PredictionString']
    data4_str = data4.iloc[i]['PredictionString']
    data5_str = data5.iloc[i]['PredictionString']
    data6_str = data6.iloc[i]['PredictionString']
    
    
    data1_str = data1_str.split(' ')
    data2_str = data2_str.split(' ')
    data3_str = data3_str.split(' ')
    data4_str = data4_str.split(' ')
    data5_str = data5_str.split(' ')
    data6_str = data6_str.split(' ')
    
    
    data1_str = np.array(data1_str, int) # best1
    data2_str = np.array(data2_str, int) # best2
    data3_str = np.array(data3_str, int) # best3
    data4_str = np.array(data4_str, int) # best4
    data5_str = np.array(data5_str, int) # best5
    data6_str = np.array(data6_str, int) # best5
    
    res = []
    
    for j in range(len(data1_str)):
        count_list = [data1_str[j], data2_str[j], data3_str[j], data4_str[j], data5_str[j], data6_str[j]]
        
        check_list = []
        for k in range(12):
            check_list.append(count_list.count(k))
            
        np_check_list = np.array(check_list)
        max_index = np_check_list.argmax()
        max_index_num = check_list[max_index]
        
        if check_list.count(max_index_num) > 1:
            res.append(data1_str[j])
        else:
            res.append(max_index)
            
    res = np.array(res)
    
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in res)}, ignore_index=True)

In [123]:
# submission.csv로 저장
submission.to_csv("/opt/ml/submission/" + "ensemble_hs2" + ".csv", index=False)

In [ ]:
# 쓰지는 않았으나 voting 함수화 작업 코드
pred1 = np.random.randint(0, 11, size=(10, 64, 64, 1))
pred2 = np.random.randint(0, 11, size=(10, 64, 64, 1))
pred3 = np.random.randint(0, 11, size=(10, 64, 64, 1))
pred4 = np.random.randint(0, 11, size=(10, 64, 64, 1))
pred5 = np.random.randint(0, 11, size=(10, 64, 64, 1))

predictions = np.concatenate([pred1, pred2, pred3, pred4, pred5], axis=3)

print(predictions.shape)

def vote_seg_map(prediction: np.ndarray, mode: str='hard'):
    return np.asarray([np.bincount(prediction[r, c]).argmax() for r in range(prediction.shape[0]) for c in range(prediction.shape[1])]).reshape(prediction.shape[:2])

# DEBUG:
# for pred in predictions[:1]:
#     plt.imshow(vote_seg_map(pred))
#     plt.colorbar()
#     plt.show()

hard_vote_result = list(map(vote_seg_map, predictions))

print(len(hard_vote_result))
print(hard_vote_result[0].dtype, hard_vote_result[0].shape)